In [1]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
!pip install tensorflow-gpu

     |████████████████████████████████| 489.6 MB 24 kB/s 


In [1]:
! nvidia-smi

Wed Dec  1 07:35:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from tensorflow.keras.layers import Input ,Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob




import os
import shutil 
import glob

In [2]:
#resize all images to this 
IMAGE_SIZE= [224,224]
train_data='/content/drive/MyDrive/Train_data'
test_data='/content/drive/MyDrive/Test_data'

In [3]:
#import the Inception V3 library as shown below and add preprocessing layer to the front of VGG
#here we will be using imagenet weights 
inception= InceptionV3(input_shape=IMAGE_SIZE + [3] ,weights='imagenet',include_top=False)

87924736/87910968 [==============================] - 1s 0us/step


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#dont train existing weights
for layer in inception.layers:
  layer.trainable = False
  
  from glob import glob
folders=glob('/content/drive/MyDrive/Train_data/*')


In [6]:
#our Layers -you can add more if   you want
x=Flatten()(inception.output)

In [7]:
prediction=Dense(len(folders),activation='softmax')(x)

In [8]:
#create a model object
model=Model(inputs=inception.input,outputs=prediction)

In [9]:
#view the structure of the model 
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [20]:
#tell the model what cost and optimization method used 
model.compile(loss=
              'categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])


In [31]:
#use the image Data generator to import the images from dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

In [32]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [33]:
#make sure you provide the same target size as initialied for the image size
training_set=train_datagen.flow_from_directory('/content/drive/MyDrive/Train_data',
                                               target_size=(224,224),
                                               batch_size=16,
                                               class_mode='categorical')

Found 1394 images belonging to 3 classes.


In [34]:
from keras.preprocessing.image import ImageDataGenerator
test_set=test_datagen.flow_from_directory('/content/drive/MyDrive/Test_data',
                                               target_size=(224,224),
                                               batch_size=16,
                                               class_mode='categorical')

Found 121 images belonging to 3 classes.


In [36]:
#fit the model
#run the cell.It will take some time to execute 
r=model.fit_generator(
    training_set,validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
88/88 [==============================] - 340s 4s/step - loss: 1.2681 - accuracy: 0.8580 - val_loss: 77.4915 - val_accuracy: 0.3471
Epoch 2/10
88/88 [==============================] - 35s 401ms/step - loss: 0.6835 - accuracy: 0.9319 - val_loss: 81.1561 - val_accuracy: 0.3223
Epoch 3/10
88/88 [==============================] - 35s 396ms/step - loss: 0.2498 - accuracy: 0.9670 - val_loss: 81.0196 - val_accuracy: 0.3306
Epoch 4/10
88/88 [==============================] - 35s 398ms/step - loss: 0.1417 - accuracy: 0.9835 - val_loss: 76.1120 - val_accuracy: 0.3471
Epoch 5/10
88/88 [==============================] - 35s 393ms/step - loss: 0.2360 - accuracy: 0.9677 - val_loss: 82.0232 - val_accuracy: 0.3306
Epoch 6/10
88/88 [==============================] - 35s 395ms/step - loss: 0.0987 - accuracy: 0.9842 - val_loss: 80.3107 - val_accuracy: 0.3471
Epoch 7/10
88/88 [==============================] - 35s 396ms/step - loss: 0.2253 - accuracy: 0.9785 - val_loss: 87.6580 - val_accuracy: 0